In [3]:
import pandas as pd     
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

In [5]:
import tarfile
file_path='/home/user/my-app-temp/Math (1).tar'
path = '/home/user/my-app-temp'
with tarfile.open(file_path, 'r') as tar:
    tar.extractall(path)

In [8]:
file_path = './MATH'
file_list={i:os.listdir(file_path+'/train/'+i) for i in os.listdir(file_path+'/train')}
for i,j in file_list.items():
    with open(file_path+'/train/'+i+'/'+j[0]) as f:
        data=json.load(f)
    data['label']=j[0]
    a=pd.json_normalize(data)
    a.set_index('label',inplace=True)
    for k in j[1:]:
        with open(file_path+'/train/'+i+'/'+k) as f:
            data_2=json.load(f)
        data_2['label']=k
        b=pd.json_normalize(data_2)
        b.set_index('label',inplace=True)
        a=pd.concat([a,b])
    a.to_csv('./MATH/'+'train/'+i+'/_'+i+'.csv')
    print(i+' done')

counting_and_probability done
intermediate_algebra done
geometry done
precalculus done
prealgebra done
number_theory done
algebra done


In [10]:
file_path = './MATH'
csv_list={i:file_path+'/train/'+i+'/_'+i+'.csv' for i in os.listdir(file_path+'/train')}
data_dict={}
for i,j in csv_list.items():
    data=pd.read_csv(j,index_col=0)
    data_dict[i]=data

data_dict['algebra'].head()

,problem,level,type,solution
label,,,,
84.json,Find $y$ if $\dfrac{y^2 - 9y + 8}{y-1} + \dfra...,Level 4,Algebra,We can factor the quadratics in the numerators...
467.json,Let $f(x) = 3x + 3$ and $g(x) = 4x + 3.$ What ...,Level 2,Algebra,"We first find that $f(2) = 9.$ Therefore, $g(f..."
1651.json,$3^n = 3 \cdot 9^3 \cdot 81^2$. What is the va...,Level 2,Algebra,We want to write everything in terms of powers...
764.json,What is the remainder when the sum of the firs...,Level 3,Algebra,"For all $n$, $1 + 2 + \dots + n = n(n + 1)/2$,..."
1938.json,Find the product of all positive integral valu...,Level 5,Algebra,"First we note that since $n^2-35n = n(n-35)$, ..."


In [16]:
import re
'''
target_char='$'
def problem_tokenizer(problem):
    p=re.search(r'\$.*?\$',problem)
    equations = re.findall(r'\\[.*?\\]', problem)
    if p:
        return p.group(),equations
    else:
        return None
'''
def preprocess_latex(latex_content):
    # This regular expression will match both inline ($...$) and display (\[...\] or $$...$$) LaTeX math expressions
    latex_math_re = re.compile(r'(\$\$?[\s\S]+?\$\$?)|(\\[.*?\\])')
    
    # Find all LaTeX math expressions in the content
    matches = latex_math_re.findall(latex_content)
    
    # Flatten the tuple results and remove None
    matches = [m[0] or m[1] for m in matches]
    
    # Preprocess each LaTeX math expression as needed
    preprocessed_expressions = []
    for match in matches:
        # Example preprocessing step: stripping whitespace
        cleaned = match.strip()
        preprocessed_expressions.append(cleaned)
    
    return preprocessed_expressions

In [12]:
%pip install -q pylatexenc 
from pylatexenc.latex2text import LatexNodes2Text

def preprocess_latex(latex_content):
    
    converter = LatexNodes2Text()
    plain_text = converter.latex_to_text(latex_content)
    preprocessed_text = plain_text.strip()
    
    return preprocessed_text


Note: you may need to restart the kernel to use updated packages.


In [4]:
for i,j in data_dict.items():
    j['problem_text']=j['problem'].apply(preprocess_latex)
    j['solution_text']=j['solution'].apply(preprocess_latex)


In [18]:
data_dict['prealgebra']

,problem,level,type,solution,problem_text,solution_text
label,,,,,,
729.json,Margaret started a stamp collection. She colle...,Level 2,Prealgebra,"Consider the sequence 8, 16, 24, $\ldots$ of t...",Margaret started a stamp collection. She colle...,"Consider the sequence 8, 16, 24, … of the numb..."
683.json,Rosie can make two pies out of nine apples. Ho...,Level 2,Prealgebra,"Since $\frac{27}{9} =3$, Rosie has 3 times the...",Rosie can make two pies out of nine apples. Ho...,"Since 27/9 =3, Rosie has 3 times the number of..."
1659.json,The number of degrees in the measures of the i...,Level 4,Prealgebra,Suppose we list the angle measures in increasi...,The number of degrees in the measures of the i...,Suppose we list the angle measures in increasi...
396.json,Calculate: $9-8+7\times6+5-4\times3+2-1$,Level 2,Prealgebra,"By the order of operations, we perform the mul...",Calculate: 9-8+7×6+5-4×3+2-1,"By the order of operations, we perform the mul..."
816.json,What is $4+10\div2-2\cdot3$?,Level 1,Prealgebra,Recall that multiplication and division should...,What is 4+10÷2-2·3?,Recall that multiplication and division should...
...,...,...,...,...,...,...
1158.json,Voldemort bought a book for $\$5$. It was one-...,Level 2,Prealgebra,We want to know what number makes the equation...,Voldemort bought a book for $5. It was one-ten...,We want to know what number makes the equation...
497.json,Express as a common fraction in simplest form:...,Level 3,Prealgebra,"As a common fraction, $6\frac{1}{4}=\frac{24}{...",Express as a common fraction in simplest form:...,"As a common fraction, 61/4=24/4+1/4=25/4. Sinc..."
35.json,What is the value of $n$ in the equation $n + ...,Level 1,Prealgebra,$n+(n+1)+(n+2)=3n+3$. So $3n+3=9\Rightarrow n...,What is the value of n in the equation n + (n ...,n+(n+1)+(n+2)=3n+3. So 3n+3=9⇒ n=2.


In [28]:
%pip install -q tiktoken

Note: you may need to restart the kernel to use updated packages.
